In [225]:
import torch

torch.cuda.is_available()


True

In [226]:
import pandas as pd
import numpy as np

In [227]:
data=pd.read_excel('politeness_finalset_data.xlsx')

In [228]:
comment=data['comment']
rating=data['final_agreed_rating'].values

In [229]:
class_names = ['polite', 'impolite', 'neutral']

In [230]:
encoding = {
    'impolite': 0,
    'neutral': 1,
    'polite': 2,
}

In [231]:
target = [encoding[x] for x in rating]

In [232]:
#from sklearn.model_selection import train_test_split

#train_comment, test_comment, train_target, test_target= train_test_split(comment, target, test_size=0.1, random_state=125)

In [233]:
#print(test_comment.shape)

In [234]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import pickle
import math
import spacy
import heapq

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
#from gensim.matutils import softcossim
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter

from sklearn import metrics
from sklearn.model_selection import train_test_split

In [235]:
torch.cuda.is_available()

True

In [236]:
from pytorch_pretrained_bert import BertModel

In [237]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [238]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [239]:
def pad_and_truncate(sequence, maxlen, dtype='int64', padding='post', truncating='post', value=0):
    x = (np.ones(maxlen) * value).astype(dtype)
    if truncating == 'pre':
        trunc = sequence[-maxlen:]
    else:
        trunc = sequence[:maxlen]
    trunc = np.asarray(trunc, dtype=dtype)
    if padding == 'post':
        x[:len(trunc)] = trunc
    else:
        x[-len(trunc):] = trunc
    return x

In [240]:
class Tokenizer(object):
    def __init__(self, max_seq_len):
        self.max_seq_len = max_seq_len
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 1

    def fit_on_text(self, text):
        words = preprocess(text)
        for word in words:
            if word not in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1

    def text_to_sequence(self, text, reverse=False, padding='post', truncating='post'):
        words = preprocess(text)
        unknownidx = len(self.word2idx)+1
        sequence = [self.word2idx[w] if w in self.word2idx else unknownidx for w in words]
        if len(sequence) == 0:
            sequence = [0]
        if reverse:
            sequence = sequence[::-1]
        return pad_and_truncate(sequence, self.max_seq_len, padding=padding, truncating=truncating)

In [241]:
max_seq_len = 70
bert_dim = 768 
polarities_dim = 3
dropout = 0.5

In [242]:
def build_tokenizer(dataset, max_seq_len):
    tokenizer = Tokenizer(max_seq_len)
    for data in dataset:
        tokenizer.fit_on_text(data)
    return tokenizer

In [243]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/amiangshu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [244]:
tokenizer=build_tokenizer(dataset = train_comment, max_seq_len = max_seq_len)
 
word2idx = train_tokenizer.word2idx


#test_tokenizer=build_tokenizer(dataset = test_comment, max_seq_len = max_seq_len)

#test_word2idx = test_tokenizer.word2idx

In [245]:
#tokenizer = build_tokenizer(dataset = comment, max_seq_len = max_seq_len)
#word2idx = tokenizer.word2idx

In [207]:
#print(word2idx)

In [246]:
class CreateDataset(Dataset):
    def __init__(self, comments, targets, sentence_tokenizer):
        
        all_data = []
        i=0
        #c=comments[i]
        #print(c)
        for i in range(len(comments)):
            
            com = comments[i]
            #print(i)
            comment_indices = sentence_tokenizer.text_to_sequence(com)
            rating = targets[i]
            
            newdata = {
                     'comment_indices': comment_indices,
                    'rating': rating,
                     }
            all_data.append(newdata)

        self.data = all_data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [247]:
#print(test_comment)

In [255]:
total_Data = CreateDataset(comment, target, tokenizer)

trainset= total_Data[1:409] 
testset=total_Data[410:500]
val_set=  total_Data[501:588]

#test=testset(frac=0.1)
#testset= CreateDataset(comment, target, test_tokenizer)

In [256]:
len(val_set)

87

In [258]:
batch_size = 32
train_data_loader = DataLoader(dataset=trainset, batch_size=10, shuffle=True, drop_last=True)


val_data_loader = DataLoader(dataset=val_set, batch_size=10, shuffle=True, drop_last=True)


test_data_loader = DataLoader(dataset=testset, batch_size=10, shuffle=True, drop_last=True)



In [259]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [260]:
class ABSABert(nn.Module):
    def __init__(self, bert, bert_dim, polarities_dim, dropout):
        
        super(ABSABert, self).__init__()
        
        
        self.bert = bert
        self.dropout = nn.Dropout(dropout)  
        self.dense = nn.Linear(bert_dim, polarities_dim)
        
    def forward(self, review_indices):
        
        # embedding 
        text_bert_indices = review_indices        # batch_size x seq_len
        _, pooled_output = self.bert(text_bert_indices,output_all_encoded_layers=False)
        
        # text_len = torch.sum(text_bert_indices != 0, dim=-1)
        
        pooled_output = self.dropout(pooled_output)
        
        logits = self.dense(pooled_output)
        
        return logits

In [261]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [262]:
mymodel = ABSABert(bert,bert_dim,polarities_dim,dropout)
mymodel = mymodel.to(device)

In [263]:
def reset_params(mymodel):
    for child in mymodel.children():
        for p in child.parameters():
            if p.requires_grad:
                stdv = 1. / math.sqrt(p.shape[0])
                torch.nn.init.uniform_(p, a=-stdv, b=stdv)
    print('Finished')

In [264]:
# function to show parameters
def paramsshow(net):
    print(net)
    params = list(net.parameters())
    print("lenghth of parameters:",len(params))
    for name,parameters in net.named_parameters():
        print(name,':',parameters.size()) 

In [265]:
criterion = nn.CrossEntropyLoss()
logdir = 'log'
learning_rate = 0.01
l2reg = 0.01
log_step = 10
lambbda = 0.5

In [266]:
params = filter(lambda p: p.requires_grad, mymodel.parameters())
optimizer = torch.optim.Adam(params, lr= learning_rate, weight_decay=l2reg)
num_epoch = 20
reset_params(mymodel)

Finished


In [267]:
def train(model, data_loader, criterion, optimizer, log_step, num_epoch, lambbda, device):
    
    print('#########Start Training#########')
    accuracy_point = []
    global_step = 0
    
    # loop over the dataset multiple times
    for epoch in range(num_epoch):
        
        print('>' * 50)
        print('epoch:', epoch)
        
        running_loss = 0.0
        
        correct = 0
        accuracy = 0
        total = 0
        
        newaccuracy = 0
        newtotal = 0
        newcorrect = 0
        
        # switch model to training mode
        model.train()
        
        for i_batch, sample_batched in enumerate(data_loader):
            
            global_step += 1
            
            losses = []
            # switch model to training mode, clear gradient accumulators
            mymodel.train()
            optimizer.zero_grad()      
        
            comment_indices = sample_batched['comment_indices'].to(device)
            rating = sample_batched['rating'].to(device)

            output = mymodel(comment_indices) 
            
            # losses
            loss = criterion(output, rating)
            
            loss.backward()
            optimizer.step()
            
            # calculate running loss 
            running_loss += loss.item()
            
            # get accuracy 
            total += rating.size(0)
            correct += (torch.argmax(output, -1) == rating).sum().item()                       
            newcorrect = correct
            newtotal = total
            newaccuracy = 100*newcorrect/newtotal
            
            if global_step %  log_step == 0:
                
                # print loss and accyracy
                print ('[%2d, %2d] loss: %.3f accuracy: %.2f' %(epoch + 1, i_batch + 1, running_loss,newaccuracy))
                running_loss = 0.0
                newtotal = 0
                newcorrect = 0
                newaccuracy = 0
                       
        accuracy = 100 * correct / total
        print ('epoch: %d, accuracy: %.2f' %(epoch,accuracy))
        accuracy_point.append(accuracy)
        
    torch.save(mymodel, 'Bert.pkl')
    print('#########Finished Training#########')
    
    return accuracy_point   

In [268]:
accuracy_point = train(mymodel, train_data_loader, criterion, optimizer, log_step, num_epoch, lambbda, device)

#########Start Training#########
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
[ 1, 10] loss: 113.517 accuracy: 36.00
[ 1, 20] loss: 41.837 accuracy: 43.00
[ 1, 30] loss: 48.209 accuracy: 43.67
[ 1, 40] loss: 35.311 accuracy: 46.00
epoch: 0, accuracy: 46.00
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 1
[ 2, 10] loss: 20.296 accuracy: 49.00
[ 2, 20] loss: 13.204 accuracy: 44.50
[ 2, 30] loss: 11.133 accuracy: 45.33
[ 2, 40] loss: 14.261 accuracy: 45.25
epoch: 1, accuracy: 45.25
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 2
[ 3, 10] loss: 10.432 accuracy: 44.00
[ 3, 20] loss: 11.707 accuracy: 50.00
[ 3, 30] loss: 10.025 accuracy: 49.00
[ 3, 40] loss: 11.966 accuracy: 48.00
epoch: 2, accuracy: 48.00
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 3
[ 4, 10] loss: 9.535 accuracy: 51.00
[ 4, 20] loss: 10.107 accuracy: 56.00
[ 4, 30] loss: 11.485 accuracy: 51.33
[ 4, 40] loss: 9.298 accuracy: 50.25
epoch: 3, accuracy: 50.25
>>>>>>>>>>>>>>>>

/home/amiangshu/anaconda3/envs/polite/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ABSABert. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#########Finished Training#########


In [269]:
def evaluate_acc_f1(test_data_loader,model):
    n_test_correct, n_test_total = 0, 0
    t_targets_all, t_outputs_all = None, None
    
    # switch model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        for t_batch, t_sample_batched in enumerate(test_data_loader):
            t_sentence = t_sample_batched['comment_indices'].to(device)
            
            #print(t_sentence)
            t_targets = t_sample_batched['rating'].to(device)
        
            t_overall = model(t_sentence) 
            
            n_test_correct += (torch.argmax(t_overall, -1) == t_targets).sum().item()
            n_test_total += len(t_overall)

            if t_targets_all is None:
                t_targets_all = t_targets
                t_outputs_all = t_overall
            else:
                t_targets_all = torch.cat((t_targets_all, t_targets), dim=0)
                t_outputs_all = torch.cat((t_outputs_all, t_overall), dim=0)

        test_acc = n_test_correct / n_test_total
        f1 = metrics.f1_score(t_targets_all.cpu(), torch.argmax(t_outputs_all, -1).cpu(), labels=[0, 1, 2], average='macro')
    
        print ('test_acc: %.4f, f1: %.4f' %(test_acc,f1))

In [270]:
newmodel = torch.load('Bert.pkl')


In [271]:
evaluate_acc_f1(test_data_loader,newmodel)


test_acc: 0.5000, f1: 0.2222
